In [1]:
!pip -q install diffusers transformers scipy mediapy deep_translator gtts langdetect pytextrank sentence-transformers sklearn wget simpletransformers sentencepiece

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from deep_translator import GoogleTranslator
def translateSentence(sentence,lang='en'):
  return GoogleTranslator('auto',lang).translate(sentence)

In [4]:
from gtts import gTTS
from langdetect import detect, DetectorFactory
def TextToSpeech(text):
  DetectorFactory.seed = 0
  language=detect(text)
  print(language)
  try:
    ttsObj = gTTS(text=text, lang=language, slow=False)
  except:
     ttsObj = gTTS(text=text, lang='ar', slow=False)
  ttsObj.save(f"test.mp3")

In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk import sent_tokenize

from sentence_transformers import SentenceTransformer
import time


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [7]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline
model_name = "google/pegasus-xsum"
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
def summarization(text):
# Create tokens
  tokens = pegasus_tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
  encoded_summary = pegasus_model.generate(**tokens)
# Decode summarized text
  decoded_summary = pegasus_tokenizer.decode(
      encoded_summary[0],
      skip_special_tokens=True
)
  summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)
  summary = summarizer(text, min_length=30, max_length=150)
  return summary[0]["summary_text"]


In [8]:
def process_bert_similarity(base_document,documents):
  # This will download and load the pretrained model offered by UKPLab.
  model = SentenceTransformer('bert-base-nli-mean-tokens')

	# Although it is not explicitly stated in the official document of sentence transformer, the original BERT is meant for a shorter sentence. We will feed the model by sentences instead of the whole documents.
  sentences = sent_tokenize(base_document)
  base_embeddings_sentences = model.encode(sentences)
  base_embeddings = np.mean(np.array(base_embeddings_sentences), axis=0)
  
  vectors = []
  for i, document in enumerate(documents):
    sentences = sent_tokenize(document)
    embeddings_sentences = model.encode(sentences)
    embeddings = np.mean(np.array(embeddings_sentences), axis=0)
    vectors.append(embeddings)
    #print("making vector at index:", i)
  scores = cosine_similarity([base_embeddings], vectors).flatten()

  highest_score = 0
  highest_score_index = 0
  score_dict={}
  for i, score in enumerate(scores):
        score_dict[score]=i
  score_dict=sorted(score_dict.items(), key=lambda x: x[0])
  return  score_dict[-10:]
  most_similar_document = documents[highest_score_index]
  print("---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")
  print(f"Most similar document: {most_similar_document} \n\n\nThe Score: {highest_score}")

In [9]:
import pandas as pd
data = pd.read_csv('APOD.csv')

In [10]:
docs=data.explanation.tolist()

In [ ]:
text='Ich möchte auf dem Mars leben'
DetectorFactory.seed = 0
language=detect(text)
eng_text=translateSentence(text)
scores_dict=process_bert_similarity(eng_text,docs[:60])


In [ ]:
TextToSpeech(translateSentence(docs[scores_dict[-1][1]],language))

In [ ]:
import wget
files=[]
for i in scores_dict:
  files.append(wget.download(data.url[i[1]]))

In [ ]:
for i in range(len(docs)):
  docs[i] = summarization(docs[i])

In [16]:
docs[30]

'The largest mountain on Mars, Olympus Mons, is pictured here in this Viking Orbiter image from 1995. Olympus Mons is a shield volcano nearly 15 miles high and over 300 miles wide at its base.'